In [1]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib.patches import Circle
from sklearn import metrics
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from PIL import Image, ImageOps
import tensorflow as tf
import sys
import cv2
import torchvision.transforms as T
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import os
from torchvision.io import read_image

# Load dataset

In [2]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file, header=None)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
            
    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        #image = torch.from_numpy(np.load(img_path + ".npy").reshape(3,224,224))
        image = read_image(img_path)
        label = torch.Tensor([self.img_labels.iloc[idx, 1].astype(np.float32)])
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

num_epochs = 50
batch_size = 32
learning_rate = 0.0001       
    
dataset = CustomImageDataset('data - Copy.csv', 'lineImages/')    
    
dataset = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Create model

In [3]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,9,(3,3), padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(9,18,(3,3), padding=1)
        self.conv3 = nn.Conv2d(18,36,(3,3), padding=1)
        self.conv4 = nn.Conv2d(36,64,(3,3), padding=1)
        self.conv5 = nn.Conv2d(64,128,(3,3), padding=1)
        self.fc1 = nn.Linear(100352,256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x, test):
#         if test == False:
#             plt.imshow(x[0].permute(1,2,0))
#             plt.show()
#         else:
#             plt.imshow(x.permute(1,2,0))
#             plt.show()
            
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = torch.relu(self.conv4(x))
        x = torch.relu(self.conv5(x))
        
        if test == False:
            x = torch.flatten(x,1) # flatten all dimensions except batch
        else:
            x = torch.flatten(x)
        #x = torch.flatten(x)    
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        return x


net = Net().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [4]:
def scaleImage(img):
    q = img.numpy()
    q = q/255
    q = torch.from_numpy(q)
    return q

# Train model

In [5]:
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, (images,labels) in enumerate(dataset):
        
        optimizer.zero_grad()
        images = scaleImage(images)
        #labels = scaleLabel(labels)
        
        images = images.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        
        # forward + backward + optimize
        
        
        outputs = net(images.float(), False)
        
        labels = labels.unsqueeze(1)
        #print(str(labels) + " " + str(outputs))
        loss = criterion(outputs.unsqueeze(1), labels)
        #print(loss)
        loss.backward()
        optimizer.step()

        
        
        # print statistics
        running_loss += loss.item()
        if i % 10 == 0:    # print every 10 mini-batches
            print(f'[Epoch: {epoch + 1}, Tuple: {i + 1:5d}] loss: {running_loss / 9:.3f}')
            running_loss = 0.0

print('Finished Training')

[Epoch: 1, Tuple:     1] loss: 0.013
[Epoch: 2, Tuple:     1] loss: 0.013
[Epoch: 3, Tuple:     1] loss: 0.015
[Epoch: 4, Tuple:     1] loss: 0.011
[Epoch: 5, Tuple:     1] loss: 0.014
[Epoch: 6, Tuple:     1] loss: 0.014
[Epoch: 7, Tuple:     1] loss: 0.013
[Epoch: 8, Tuple:     1] loss: 0.017
[Epoch: 9, Tuple:     1] loss: 0.011
[Epoch: 10, Tuple:     1] loss: 0.014
[Epoch: 11, Tuple:     1] loss: 0.010
[Epoch: 12, Tuple:     1] loss: 0.016
[Epoch: 13, Tuple:     1] loss: 0.010
[Epoch: 14, Tuple:     1] loss: 0.010
[Epoch: 15, Tuple:     1] loss: 0.009
[Epoch: 16, Tuple:     1] loss: 0.006
[Epoch: 17, Tuple:     1] loss: 0.005
[Epoch: 18, Tuple:     1] loss: 0.008
[Epoch: 19, Tuple:     1] loss: 0.005
[Epoch: 20, Tuple:     1] loss: 0.006
[Epoch: 21, Tuple:     1] loss: 0.004
[Epoch: 22, Tuple:     1] loss: 0.003
[Epoch: 23, Tuple:     1] loss: 0.004
[Epoch: 24, Tuple:     1] loss: 0.003
[Epoch: 25, Tuple:     1] loss: 0.004
[Epoch: 26, Tuple:     1] loss: 0.001
[Epoch: 27, Tuple:   

In [6]:
PATH = './jetson50.pth'
torch.save(net.state_dict(), PATH)

# Loading the trained network
#net.load_state_dict(torch.load(PATH))

In [7]:
correct = 0
total = 0
print("Actual: Prediction:")
dataset = CustomImageDataset('data - Copy.csv', 'lineImages/')    
#net.eval()
with torch.no_grad():
    for data in dataset:
        images, labels = data
        
        images = images.to(device)
        labels = labels.to(device)
        
        images = scaleImage(images)
        outputs = net(images.float(), True)
        predicted = outputs.data
        
        total += labels.size(0)
        correct += (outputs.data == labels).sum().item()
        print(str(np.asarray(labels)) + ", " + str(np.asarray(outputs.data)))

Actual: Prediction:
[0.], [0.00374356]
[0.], [-0.0207608]
[0.], [-0.03665822]
[0.], [0.03407381]
[0.5], [0.54775655]
[-0.25], [-0.23333643]
[-0.25], [-0.2919544]
[-0.25], [-0.2700745]
[0.], [0.00547]
[0.], [0.00572329]
[0.1], [0.15498313]
[0.], [-0.00269682]
[0.05], [0.07751779]
[-0.1], [-0.10059048]
[-0.5], [-0.52059096]
[-0.05], [-0.0747688]
[0.5], [0.4947714]
[0.25], [0.29362497]
[-0.1], [-0.10046734]
[-0.05], [-0.05026007]
[0.], [0.02818231]
[0.025], [0.1110718]
[-0.5], [-0.82914406]
[0.1], [0.14294569]
[-0.05], [-0.0769152]
[-0.7], [-0.6771695]
[0.], [0.06072842]
[0.], [0.03430657]
[0.5], [0.5539761]
[1.], [0.96105725]
[1.], [0.87164843]
[1.], [0.9536912]
[0.5], [0.5447535]
[-1.], [-0.8953686]
[0.025], [0.02438519]
[-0.25], [-0.23602971]
[-0.3], [-0.3217786]
[0.05], [0.0621318]
[-0.5], [-0.63017875]
[0.05], [0.06007835]
[0.05], [0.15766867]
[0.025], [0.01219025]
[0.2], [0.25183678]
[0.], [0.01672929]
[0.7], [0.7178586]
[-0.5], [-0.4698207]
[0.], [-0.00836034]
[-0.025], [-0.0219920